In [1]:
# Instalei as bibliotecas necessárias para processamento de PDF e integração com a OpenAI
%pip install -qU pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importei as bibliotecas necessárias
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [3]:
# Carreguei a chave da API do meu arquivo .env
load_dotenv()

True

In [4]:
# Defini o caminho do arquivo PDF para ser processado
file_path = "Mapeamento de Ecosssistema de Startups 2023.pdf"

In [5]:
# Carreguei o conteúdo do PDF
loader = PyPDFLoader(file_path)
docs = loader.load()

In [6]:
# Inicializei o modelo de linguagem GPT-4, ajustado para tarefas específicas
llm = ChatOpenAI(model="gpt-4o")

In [7]:
# Dividi o documento em partes menores para facilitar o processamento e a recuperação de contexto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [8]:
# Criei um armazenamento vetorial para guardar as representações vetoriais dos pedaços do documento
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

In [9]:
# Configurei o armazenamento vetorial como um "retriever" para buscar trechos relevantes do documento
retriever = vectorstore.as_retriever()

In [10]:
# Carreguei um modelo de prompt específico para RAG (Recuperação Aumentada por Geração) da biblioteca de prompts
prompt = hub.pull("rlm/rag-prompt")

In [11]:
# Defini uma função para formatar os documentos em uma sequência de conteúdo de página
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
# Configurei a cadeia de RAG: usando o contexto de busca no retriever e o modelo de linguagem para gerar respostas
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# Executei a cadeia de RAG para responder a uma pergunta específica sobre o faturamento das startups
rag_chain.invoke("Qual foi o faturamento médio das Startups no ano de 2023?")

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Forbidden"}')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Forbidden"}')


'O faturamento médio das startups no ano de 2023 foi de R$ 876.034,31.'

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Forbidden"}')
